# Test the three models

In [1]:
import keras
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Add, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History

import datetime
from time import gmtime, strftime
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sys
import os

from mapgenlib.res_unit import build_res_unet
from mapgenlib.losses_extend import dice_coef_loss, iou_loss, binary_focal_loss
from mapgenlib.read_write import check_and_create, load_data, load_batch, save_hist, save_hist_local

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class GAN4MapGen(): # Based on u-net, residual u-net and pix2pix
    # Reference: https://github.com/eriklindernoren/Keras-GAN/blob/master/pix2pix/pix2pix.py
    
    def __init__(self, img_rows, img_cols):

        # Input shape
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.clip_value = 0.01
        
        # Configure data loader
        self.dataset_name = 'mapgen'

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)
        
        # Calculate output shape of D (PatchGAN) better version
        self.patch_size = 32
        self.nb_patches = int((self.img_rows / self.patch_size) * (self.img_cols / self.patch_size))
        self.patch_gan_dim = (self.patch_size, self.patch_size, self.channels)
        
        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64
        optimizer = Adam(0.0004)
        
        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        #self.generator = self.build_generator() # Old generator from 
        self.generator = self.build_res_unet_generator()

        # Input images and their conditioning images
        #img_A = Input(shape=self.img_shape) # Target
        img_B = Input(shape=self.img_shape) # Input

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)
        
        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator() # 1.Version
        self.discriminator = self.build_discriminator_patchgan()
        
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        #self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        #valid = self.discriminator([fake_A, img_B])
        #self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        
        valid = self.discriminator([fake_A])
        self.combined = Model(inputs= img_B, outputs=[valid, fake_A])
        
        # Original Pix2Pix - low weight for discriminator
        self.combined.compile(loss=['mse', 'mae'], #['mse', 'mae'] original
                              loss_weights=[1, 100], # [1, 100] original
                              optimizer=optimizer, metrics=['accuracy'])

    
    def build_res_unet_generator(self):
        return build_res_unet(self.img_shape)
        
        """Residual U-Net Generator"""
        """
        inputs = Input(shape=self.img_shape)
        to_decoder = encoder(inputs)
        path = res_block(to_decoder[2], [256, 256], [(2, 2), (1, 1)], increase = True) # 3x
        path = res_block(path, [256, 256], [(1, 1), (1, 1)]) # Number of block of bottleneck = 1
        path = res_block(path, [256, 256], [(1, 1), (1, 1)]) # Try to add one 2019.01.14, achieved best result ever
        path = decoder(path, from_encoder=to_decoder)
        path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path) 
        return Model(input=inputs, output=path)
        """
        
    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)
    
    def build_discriminator_simple(self):

        def d_block(layer_input, filters, strides=1, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        # Input img
        d0 = Input(shape=self.hr_shape)

        d1 = d_block(d0, self.df, bn=False)
        d2 = d_block(d1, self.df, strides=2)
        d3 = d_block(d2, self.df*2)
        d4 = d_block(d3, self.df*2, strides=2)
        d5 = d_block(d4, self.df*4)
        d6 = d_block(d5, self.df*4, strides=2)
        d7 = d_block(d6, self.df*8)
        d8 = d_block(d7, self.df*8, strides=2)

        d9 = Dense(self.df*16)(d8)
        d10 = LeakyReLU(alpha=0.2)(d9)
        validity = Dense(1, activation='sigmoid')(d10)

        return Model(d0, validity)
    
    def build_discriminator_patchgan(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)
        
    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=3, bn=True): # Chnaged here for the order of bn and activation
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            d = Activation(activation='relu')(d)
            #d = LeakyReLU(alpha=0.2)(d)
            return d

        img_A = Input(shape=self.img_shape)
        
        # !!!! This step was deleted because the condition do not help in this case
        #img_B = Input(shape=self.img_shape)
        ## Concatenate image and conditioning image by channels to produce input
        #combined_imgs = Concatenate(axis=-1)([img_A, img_B])
        #d1 = d_layer(combined_imgs, self.df, bn=False)
        
        d1 = d_layer(img_A, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=3, strides=1, padding='same')(d4)

        return Model([img_A], validity)
    
    def train_generator_only(self, x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath):
        
        start_time = datetime.datetime.now()
        
        gen = self.build_res_unet_generator()
        optimizer = Adam(0.0004) # Current best
        
        # Loss function finished
        gen.compile(loss='mse', optimizer=optimizer, metrics=['accuracy']) # mse better than mae

        data_gen_args = dict(rotation_range=180.)
        image_datagen = ImageDataGenerator(**data_gen_args)
        mask_datagen = ImageDataGenerator(**data_gen_args)
        
        seed = 2
        BATCH_SIZE = 16 # Proper for 128x128
        result_generator = zip(image_datagen.flow(x_train_sim, batch_size=BATCH_SIZE, seed=seed), 
                               mask_datagen.flow(y_train_sim, batch_size=BATCH_SIZE, seed=seed))
        
        History1 = History()
        hist1 = gen.fit_generator(result_generator,
                                  epochs = 100,
                                  steps_per_epoch=2000,
                                  verbose=1,
                                  shuffle=True,
                                  callbacks=[History1, 
                                             EarlyStopping(patience=4), 
                                             ReduceLROnPlateau(patience = 3, verbose = 0),
                                             ModelCheckpoint(outPath + "weights.h5", 
                                                             save_best_only = True, 
                                                             save_weights_only = False)],
                                  validation_data=(x_test_sim, y_test_sim))
        save_hist(History1, outPath)
        
    
    def train(self, x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        total_samples = len(x_train_sim)
        ids = np.arange(total_samples)
        np.random.shuffle(ids)
        n_batches = int(total_samples / batch_size)
        
        train_acc = []
        train_loss = []
        valid_acc = []
        valid_loss = []
        
        patience = 5
        
        for epoch in range(epochs):
            
            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(x_train_sim, y_train_sim, batch_size)):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                fake_A = self.generator.predict(imgs_B)
                
                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                
                ## Clip critic weights
                #for l in self.discriminator.layers:
                #    weights = l.get_weights()
                #    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                #    l.set_weights(weights)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                self.discriminator.trainable = False
                g_loss = self.combined.train_on_batch(imgs_B, [valid, imgs_A])
                self.discriminator.trainable = True
                
                if batch_i % sample_interval == 0:
                    elapsed_time = datetime.datetime.now() - start_time
                    print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                elapsed_time))    
                
            # Plot the progress
            if epoch >= 0:
                    
                    elapsed_time = datetime.datetime.now() - start_time
                    
                    valid_test = np.ones((len(x_test_sim),) + self.disc_patch)
                    t_loss = self.combined.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
                    
                    print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] [Test loss&acc: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                100*t_loss[2], 100*t_loss[3], 100*t_loss[4],
                                                                                elapsed_time))    
                    
                    train_loss.append(g_loss[2])
                    train_acc.append(g_loss[4])
                    valid_loss.append(t_loss[2])
                    valid_acc.append(t_loss[4])

                    waited = len(valid_loss) - 1 - np.argmin(valid_loss)
                    print('waited for', waited, valid_loss)
                    
                    if waited == 0:
                        self.generator.save(outPath + 'model_epoch'+ str(epoch) +'.h5')   
                        
                    if waited > patience:
                        break
            
        return train_acc, train_loss, valid_acc, valid_loss

In [3]:
# Order the image dimension acc. to TensorFlow (batc_hsize, rows, cols, channels)
K.set_image_dim_ordering('tf')

#scale = 25
#p_size_1 = 128 # Compared with 256, which larger may generate round corners

scale = 10
p_size_1 = 256 # Larger inception area

trainPath = r"../tmp_data/data_feng_"+str(p_size_1)+"/geb" + str(scale) +  "/"
x_train_sim, y_train_sim, x_test_sim, y_test_sim = load_data(trainPath)

input_shape1 = (None, None, 1)
print('Input Shape of the trains', x_train_sim.shape)
print('Input Shape of the tests', x_test_sim.shape)

Input Shape of the trains (10268, 256, 256, 1)
Input Shape of the tests (1140, 256, 256, 1)


### Train residual unet

In [ ]:
############ Path Setting ##############
outPath = r"../tmp_results/predictions/"
timestr = strftime("U"+str(p_size_1)+"_%Y-%m-%d %H-%M-%S", gmtime())
outPath = outPath + timestr + '_' + str(scale)+ "/"
check_and_create(outPath)
print(outPath)

../tmp_results/predictions/U256_2019-03-21 14-38-55_10/


In [ ]:
gan = GAN4MapGen(p_size_1, p_size_1)
gan.train_generator_only(x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath)

/notebooks/tmp/MapGen/mapgenlib/res_unit.py:96: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  return Model(input=inputs, output=path)


Epoch 1/100
2000/2000 [==============================] - 871s 435ms/step - loss: 0.0039 - acc: 0.9762 - val_loss: 0.0044 - val_acc: 0.9952
Epoch 2/100
1783/2000 [=========================>....] - ETA: 1:31 - loss: 0.0029 - acc: 0.9772

### Train residual unet + PatchGAN